In [ ]:
from sklearn.metrics import confusion_matrix, recall_score ,precision_score , fbeta_score ,cohen_kappa_score  ,log_loss , roc_auc_score
# import scikitplot
# from scikitplot.metrics import plot_roc
import matplotlib.pyplot as plt
# from matplotlib.pyplot 

def performance_measures(y_true , y_pred_pos):
  #confusion matrix
  print("Confusion matrix")
  y_pred_class = y_pred_pos 
  cm = confusion_matrix(y_true, y_pred_class)
  print(cm)
  tn, fp, fn, tp = cm.ravel()
  
  #recall score
  print("Recall score = " , recall_score(y_true, y_pred_class))

  #precision score
  print("Precision score = " , precision_score(y_true, y_pred_class))

  #beta scores
  print("F1 score =",fbeta_score(y_true, y_pred_class, 1))
  print("F2 score =", fbeta_score(y_true, y_pred_class, 2))

  #cohen-capa
  print("Cohen capa score = " , cohen_kappa_score(y_true, y_pred_class))

  #log-loss
  print("Log - loss = ", log_loss(y_true, y_pred_pos))

  #roc_auc
  print("Roc_auc score=", roc_auc_score(y_true, y_pred_pos))

  # #roc-curve
  # fig, ax = plt.subplots()
  # plot_roc(y_true, y_pred_pos, ax=ax)

  


In [ ]:
from google.colab import files
file = files.upload()

Saving dev-access.csv to dev-access.csv


In [ ]:
import pandas as pd
df= pd.read_csv('dev-access.csv',engine='python', quotechar='|', header=None)


In [ ]:
import sys
import os
import json
import pandas
import numpy
import optparse
import numpy as np
import keras
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout ,SimpleRNN , GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict


In [ ]:
def preprocess():
    dataframe = df.copy()
    dataset = dataframe.sample(frac=1).values

    # Preprocess dataset
    X = np.asarray(dataset[:,0])
    Y = np.asarray(dataset[:,1])

    for index, item in enumerate(X):
        # Quick hack to space out json elements
        reqJson = json.loads(item, object_pairs_hook=OrderedDict)
        del reqJson['timestamp']
        del reqJson['headers']
        del reqJson['source']
        del reqJson['route']
        del reqJson['responsePayload']
        X[index] = json.dumps(reqJson, separators=(',', ':'))

    # tokenizer = Tokenizer(filters='\t\n', char_level=True)
    # tokenizer.fit_on_texts(X)

    # Extract and save word dictionary
    # word_dict_file = 'build/word-dictionary.json'

    # if not os.path.exists(os.path.dirname(word_dict_file)):
    #     os.makedirs(os.path.dirname(word_dict_file))

    # with open(word_dict_file, 'w') as outfile:
    #     json.dump(tokenizer.word_index, outfile, ensure_ascii=False)

    # num_words = len(tokenizer.word_index)+1
    # X = tokenizer.texts_to_sequences(X)

    max_log_length = 1024
    train_size = int(len(dataset) * .75)

    X_processed = X
    X_train, X_test = np.asarray(X_processed[0:train_size]), np.asarray(X_processed[train_size:len(X_processed)])
    Y_train, Y_test = np.asarray(Y[0:train_size]), np.asarray(Y[train_size:len(Y)])
    # X_train = np.expand_dims(X_train, -1)
    # Y_train = np.expand_dims(Y_train, -1)
    # X_train=np.asarray(X_train).astype(np.float32)
    Y_train = np.asarray(Y_train).astype(np.float32)
    # # X_test = np.asarray(X_test).astype(np.float32)
    Y_test = np.asarray(Y_test).astype(np.float32)
    return X_train,X_test , Y_train, Y_test



In [ ]:
X_train,X_test , Y_train, Y_test = preprocess()

In [ ]:
X_train[0]

'{"method":"post","query":{},"path":"/login","statusCode":401,"requestPayload":{"username":"DROP","password":"worldburn123123432"}}'

In [ ]:
Y_train[0]

1.0

In [ ]:
!pip install transformers
!pip install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 1.9MB 18.4MB/s 
     |████████████████████████████████| 3.2MB 39.4MB/s 
     |████████████████████████████████| 890kB 55.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=92c8f9d47ebd599a5123e5c8437f8468848056bad707f1ed9fa747b675d7d3f1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.3.3)
     |████████████████████████████████| 215kB 18.3MB/s 
     |████████████████████████████████| 7.5MB 12.5MB/s 
     |████████████████████████████████| 2.0MB 46.1MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 317kB 58.4MB/s 
     |████████████████████████████████| 1.2MB 56.2MB/s 
     |████████████████████████████████| 112kB 56.6

In [ ]:
train_df = pd.DataFrame({
    'text': X_train,
    'label':Y_train
})

print(train_df.head())

eval_df = pd.DataFrame({
    'text': X_test,
    'label':Y_test
})

                                                text  label
0  {"method":"post","query":{},"path":"/login","s...    1.0
1  {"method":"get","query":{"query":"aj"},"path":...    0.0
2  {"method":"post","query":{},"path":"/login","s...    1.0
3  {"method":"post","query":{},"path":"/login","s...    0.0
4  {"method":"get","query":{"query":"civegili"},"...    0.0


In [ ]:
len(train_df)

20079

In [ ]:
from simpletransformers.classification import ClassificationModel
# import 


# Create a TransformerModel
model =ClassificationModel('bert', 'bert-base-uncased', use_cuda=True,num_labels=2)


# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2510 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running Evaluation:   0%|          | 0/837 [00:00<?, ?it/s]

In [ ]:
result

{'auprc': 1.0000000000000002,
 'auroc': 0.9999999999999999,
 'eval_loss': 0.0013714823955889581,
 'fn': 1,
 'fp': 0,
 'mcc': 0.999701261438526,
 'tn': 3329,
 'tp': 3364}

In [ ]:
model_outputs

array([[ 6.51953125, -5.17578125],
       [ 6.51953125, -5.17578125],
       [ 6.51953125, -5.15234375],
       ...,
       [ 6.37890625, -5.15234375],
       [ 6.5       , -5.10546875],
       [ 6.53125   , -5.1640625 ]])

In [ ]:
preds= model.predict(X_test)[0].tolist()

  0%|          | 0/837 [00:00<?, ?it/s]

In [ ]:
model.predict([X_test[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

(array([0]), array([[ 6.51953125, -5.17578125]]))

In [ ]:
# X_test[3]

'{"method":"get","query":{"query":"Pressure Washers"},"path":"/search","statusCode":200,"requestPayload":null}'

In [ ]:
performance_measures(Y_test , preds)

Confusion matrix
[[3329    0]
 [   1 3364]]
Recall score =  0.999702823179792
Precision score =  1.0
F1 score = 0.9998513895080993
F2 score = 0.9997622444127438
Cohen capa score =  0.9997012168161639
Log - loss =  0.005159661845670357
Roc_auc score= 0.999851411589896


In [ ]:
from simpletransformers.classification import ClassificationModel
# import 


# Create a TransformerModel
# ClassificationModel()
model =ClassificationModel('roberta', 'roberta-base', use_cuda=True,num_labels=2)


# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2510 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running Evaluation:   0%|          | 0/837 [00:00<?, ?it/s]

In [ ]:
result

{'auprc': 0.9999999999999999,
 'auroc': 1.0,
 'eval_loss': 3.4656920625161356e-06,
 'fn': 0,
 'fp': 0,
 'mcc': 1.0,
 'tn': 3343,
 'tp': 3351}

In [ ]:
preds= model.predict(X_test)[0].tolist()

  0%|          | 0/837 [00:00<?, ?it/s]

In [ ]:
performance_measures(Y_test , preds)

Confusion matrix
[[3343    0]
 [   0 3351]]
Recall score =  1.0
Precision score =  1.0
F1 score = 1.0
F2 score = 1.0
Cohen capa score =  1.0
Log - loss =  9.992007221626413e-16
Roc_auc score= 1.0


In [ ]:
from simpletransformers.classification import ClassificationModel
# import 


# Create a TransformerModel
# ClassificationModel()
model =ClassificationModel('xlnet', 'xlnet-base-cased', use_cuda=True,num_labels=2)


# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2510 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe 

Running Evaluation:   0%|          | 0/837 [00:00<?, ?it/s]

In [ ]:
result

{'auprc': 0.9999999999999999,
 'auroc': 1.0,
 'eval_loss': 0.0008416683638644638,
 'fn': 0,
 'fp': 1,
 'mcc': 0.9997012494392008,
 'tn': 3319,
 'tp': 3374}

In [ ]:
preds= model.predict(X_test)[0].tolist()

  0%|          | 0/837 [00:00<?, ?it/s]

In [ ]:
performance_measures(Y_test , preds)

Confusion matrix
[[3319    1]
 [   0 3374]]
Recall score =  1.0
Precision score =  0.9997037037037036
F1 score = 0.9998518299007261
F2 score = 0.9999407266907713
Cohen capa score =  0.9997012048132539
Log - loss =  0.005159781295540418
Roc_auc score= 0.9998493975903615


In [ ]:
for i in range(len(preds)):
  if preds[i]!=Y_test[i]:
    print(X_test[i] , preds[i] , Y_test[i])

{"method":"get","query":{"query":"or"},"path":"/search","statusCode":404,"requestPayload":null} 1 0.0
